## PySpark 설치

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 631.5 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 3.8 MB/s eta 0:00:0000:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845499 sha256=e45ba44034a0a468c19b5784929605198b63c103a2a946105d11d0cd9c99070d
  Stored in directory: /home/sangwon/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


## 처리할 데이터 파일을 먼저 다운로드 받아온다

In [2]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv

--2024-06-20 01:21:51--  https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.196.162, 3.5.78.19, 52.218.236.33, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.196.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62728 (61K) [text/csv]
Saving to: ‘1800.csv’

1800.csv            100%[===================>]  61.26K   358KB/s    in 0.2s    

2024-06-20 01:21:52 (358 KB/s) - ‘1800.csv’ saved [62728/62728]



In [3]:
!ls -tl

total 232
-rw-r--r-- 1 sangwon sangwon 38154 Jun 20 01:22 PySpark_DataFrame_1.ipynb
-rw-r--r-- 1 sangwon sangwon 24719 Jun 20 01:17 PySpark_DataFrame_2.ipynb
-rw-r--r-- 1 sangwon sangwon 13964 Jun 20 01:17 PySpark_DataFrame_3.ipynb
-rw-r--r-- 1 sangwon sangwon 31613 Jun 20 01:17 PySpark_DataFrame_4.ipynb
-rw-r--r-- 1 sangwon sangwon 21447 Jun 20 01:17 PySpark_DataFrame_5.ipynb
-rw-r--r-- 1 sangwon sangwon 26539 Jun 20 01:17 PySpark_설치_및_테스트.ipynb
-rw-r--r-- 1 sangwon sangwon 62728 Apr 10  2022 1800.csv


In [4]:
!head -5 1800.csv

ITE00100554,18000101,TMAX,-75,,,E,
ITE00100554,18000101,TMIN,-148,,,E,
GM000010962,18000101,PRCP,0,,,E,
EZE00100082,18000101,TMAX,-86,,,E,
EZE00100082,18000101,TMIN,-135,,,E,


## 판다스 데이터프레임으로 처리해본다

In [5]:
import pandas as pd 

pd_df = pd.read_csv(
    "1800.csv",
    names=["stationID", "date", "measure_type", "temperature"],
    usecols=[0, 1, 2, 3]
)

In [6]:
pd_df.head()

,stationID,date,measure_type,temperature
0,ITE00100554,18000101,TMAX,-75
1,ITE00100554,18000101,TMIN,-148
2,GM000010962,18000101,PRCP,0
3,EZE00100082,18000101,TMAX,-86
4,EZE00100082,18000101,TMIN,-135


In [7]:
# Filter out all but TMIN entries
pd_minTemps = pd_df[pd_df['measure_type'] == "TMIN"]

In [8]:
pd_minTemps.head()

,stationID,date,measure_type,temperature
1,ITE00100554,18000101,TMIN,-148
4,EZE00100082,18000101,TMIN,-135
6,ITE00100554,18000102,TMIN,-125
9,EZE00100082,18000102,TMIN,-130
11,ITE00100554,18000103,TMIN,-46


In [9]:
# Select only stationID and temperature
pd_stationTemps = pd_minTemps[["stationID", "temperature"]]

In [10]:
# Aggregate to find minimum temperature for every station
pd_minTempsByStation = pd_stationTemps.groupby(["stationID"]).min("temperature")
pd_minTempsByStation.head()

,temperature
stationID,
EZE00100082,-135
ITE00100554,-148


## Spark으로 처리해본다

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #1")
conf.set("spark.master", "local[*]")

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/20 01:53:12 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/20 01:53:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 01:53:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.format("csv").load("1800.csv") # spark.read.csv("1800.csv")

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [14]:
df = spark.read.format("csv")\
    .load("1800.csv")\
    .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [15]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: string (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [16]:
df = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .load("1800.csv")\
    .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [17]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [18]:
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.types import StructType, StructField

schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)])

In [19]:
# df = spark.read.schema(schema).format("csv").load("1800.csv")
df = spark.read.schema(schema).csv("1800.csv")

In [20]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [21]:
# Filter out all but TMIN entries
minTemps = df.filter(df.measure_type == "TMIN")

In [22]:
minTemps.count()

730

In [23]:
# Column expression으로 필터링 적용
minTemps = df.where(df.measure_type == "TMIN")

In [24]:
minTemps.count()

730

In [25]:
# SQL expression으로 필터링 적용
minTemps = df.where("measure_type = 'TMIN'")

In [26]:
minTemps.count()

730

In [27]:
# Aggregate to find minimum temperature for every station
minTempsByStation = minTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [28]:
# Select only stationID and temperature
stationTemps = minTemps[["stationID", "temperature"]]

In [29]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [30]:
stationTemps = minTemps.select("stationID", "temperature")

In [31]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [32]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [33]:
# Collect, format, and print the results
results = minTempsByStation.collect()

In [34]:
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	-148.00F
EZE00100082	-135.00F


## Spark SQL로 처리해보기

In [35]:
df.createOrReplaceTempView("station1800")

In [36]:
results = spark.sql("""SELECT stationID, MIN(temperature)
FROM station1800
WHERE measure_type = 'TMIN'
GROUP BY 1""").collect()

In [37]:
# pyspark.sql.Row는 DataFrame의 레코드에 해당하며 필드별로 이름이 존재# 
for r in results:
    print(r)

Row(stationID='ITE00100554', min(temperature)=-148.0)
Row(stationID='EZE00100082', min(temperature)=-135.0)
